# Examples usage of the majority voting I/O processor

This notebook should eventually turn into documentation.

In [ ]:
from granite_io import make_backend
from granite_io.io import make_io_processor
from granite_io.io.base import ChatCompletionInputs
from granite_io.io.voting import MajorityVotingProcessor, integer_normalizer

In [ ]:
%pip install openai pandas

# Connect to a backend running on localhost.
port = 11434
url = f"http://localhost:{port}/v1"
model_name = "granite3.2:2b"
backend = make_backend(
    "openai",
    {
        "model_name": model_name,
        "openai_base_url": url,
    },
)

# Uncomment to run inference on a local Transformers backend.
#
# %pip install transformers
#
# backend =  make_backend(
#     "transformers",
#     {
#         "model_name": "ibm-granite/granite-3.2-2b-instruct",
#     },
# )

In [ ]:
# Ask a question with an integral answer
completion_inputs = ChatCompletionInputs(
    messages=[
        {
            "role": "user",
            "content": "What is 234651 + 13425?\nAnswer with just a number please.",
        }
    ],
    thinking=True,
    generate_inputs={"n": 3, "temperature": 0.8, "max_tokens": 1024},
)
completion_inputs

In [ ]:
# Run the question through the base model
base_processor = make_io_processor("Granite 3.2", backend=backend)
results = base_processor.create_chat_completion(completion_inputs)
print("Outputs from base model:")
for result_num, r in enumerate(results.results):
    print(f"{result_num + 1}: {r.next_message.content}")

In [ ]:
# Wrap the base model's I/O processor in a majority voting I/O processor.
voting_processor = MajorityVotingProcessor(
    base_processor, integer_normalizer, samples_per_completion=100
)
results = voting_processor.create_chat_completion(completion_inputs)
print("Outputs from base model augmented with majority voting:")
for result_num, r in enumerate(results.results):
    print(f"{result_num + 1}: {r.next_message.content}")

# What's the actual answer?
print(f"---------\nThe actual answer is: {234651 + 13425}")